In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import OrderedDict

from lxml import etree
from tqdm import tqdm


def read_book(fname):
    result = OrderedDict()
    
    # Extract the words
    context = etree.iterparse(fname, events=('end', ), tag=('book'))
    for event, elem in tqdm(context):
        for e in elem.iterchildren():
            if e.tag == 'id':
                t_id = e.text
                #print(t_id)
            elif e.tag == 'nass':   # text
                # sometimes the text is None (e.g., file 16, id 214)
                text = e.text
                if text is None:
                    print('None text found in {} (prev? id: {})'.format(fname, t_id))
                else:
                    # &#xd; is automatically replaced by \r, but we would like to have \n
                    text = text.replace('\r', '\n')
                    
            elif e.tag == 'page':
                page = e.text
            elif e.tag == 'part':
                part = e.text
            #print(e.tag)
        
        # ignore entries that don't have text
        if not text is None:
            result[t_id] = {'text': text, 'page': page, 'part': part}

        # make iteration over context fast and consume less memory
        #https://www.ibm.com/developerworks/xml/library/x-hiperfparse
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    
    return result

book = read_book('/home/jvdzwaan/Downloads/book.xml')
print(len(book))

In [ ]:
def read_title(fname):
    result = {}
    levels = {}
    
    # Extract the words
    context = etree.iterparse(fname, events=('end', ), tag=('title'))
    for event, elem in tqdm(context):
        for e in elem.iterchildren():
            if e.tag == 'id':
                t_id = e.text
                #print(t_id)
            elif e.tag == 'tit':
                title = e.text
                # &#xd; is automatically replaced by \r, but we would like to have \n
                title = title.replace('\r', '\n')
            elif e.tag == 'lvl':
                level = e.text
                levels[level] = None
            #print(e.tag)
        result[t_id] = {'title': title, 'level': level}

        # make iteration over context fast and consume less memory
        #https://www.ibm.com/developerworks/xml/library/x-hiperfparse
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    
    print('levels found:', levels.keys())
    return result

title = read_title('/home/jvdzwaan/Downloads/title.xml')
print(len(title))

In [ ]:
# combine book and title
# and write to file
import codecs

def combine_data(book, title, out_file):
    with codecs.open(out_file, 'w', encoding='utf-8') as f:
        for b_id, data in book.items():
            text = data['text']
            if b_id in title:
                #print(b_id)
                #print('title!')
                ti = title[b_id]['title']
                lvl = title[b_id]['level']
                #print(title[b_id]['title'], title[b_id]['level'])
                if text.strip().startswith(ti):
                    #print('title in text')
                    text = text.replace(ti, '### {} {}\n'.format(int(lvl)*'|', ti))
                else:
                    text = '### {} {}\n'.format(int(lvl)*'|', ti) + text
                    #print('title text not found', b_id)
            f.write(text.strip())
            f.write('\n')
            # sometimes the page marker information is incomplete (e.g. for book 12)
            # in that case, we don't write the marker information
            if not data['part'] is None and not data['page'] is None:
                page = data['page'].split()[0]
                part = data['part'].split()[0]
                
                try:
                    part = int(part)
                except ValueError:
                    print('Invalid part {} ({})'.format(part, out_name))
                    part = 0
                    
                try:
                    page = int(page)
                except ValueError:
                    print('Invalid page {} ({})'.format(page, out_name))
                    page = 0
                
                page_marker = 'V{:03}P{:04}'.format(part, page)
                f.write(page_marker)
                f.write('\n')
            #print('---')
combine_data(book, title, 'test.txt')

In [ ]:
import os

in_dir = '/home/jvdzwaan/Downloads/New_xml_corpus/New_xml_corpus/'
out_dir = '/home/jvdzwaan/data/adh-corpora/dawa/txt/'


for i in range(1, 217):
    out_name = '{}.txt'.format(i)
    
    d = os.path.join(in_dir, str(i))
    book_file = os.path.join(d, 'book.xml')
    title_file = os.path.join(d, 'title.xml')
    #print(book_file)
    #print(title_file)
    book = read_book(book_file)
    title = read_title(title_file)
    
    out_file = os.path.join(out_dir, out_name)
    print(out_file)
    combine_data(book, title, out_file)
    